In [4]:
from utils import datasets
from utils.parser import * 
import os 
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import tensorflow as tf 
from models import * 
from utilities import *
from train_model_utilities import * 
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [17]:
dataset_name = 'seq_FUN'
train, valid, test = initialize_dataset(dataset_name, datasets)

scalar = StandardScaler().fit(train.X)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean').fit(train.X)
train.X = scalar.transform(imputer.transform(train.X))
valid.X = scalar.transform(imputer.transform(valid.X))
test.X = scalar.transform(imputer.transform(test.X))

train.Y = tf.convert_to_tensor(train.Y, tf.float32)
valid.Y = tf.convert_to_tensor(valid.Y, tf.float32)
test.Y = tf.convert_to_tensor(test.Y, tf.float32)

In [18]:
train_data = tf.data.Dataset.from_tensor_slices(
    (train.X, train.Y)
).shuffle(buffer_size = train.X.shape[0]).batch(batch_size=64)

In [5]:
def get_structure_from_adajancency(adajancency):
    structure = np.zeros(adajancency.shape)
    g = nx.DiGraph(adajancency) # train.A is the matrix where the direct connections are stored 
    for i in range(len(adajancency)):
        ancestors = list(nx.descendants(g, i)) #here we need to use the function nx.descendants() because in the directed graph the edges have source from the descendant and point towards the ancestor 
        if ancestors:
            structure[i, ancestors] = 1
    return structure 

In [19]:
structure = get_structure_from_adajancency(train.A)
model = HMCModel(structure, 500, [2000,2000,2000], 0.7)
loss_fn = get_loss_fn_coherent(structure)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
train_epoch(model, optimizer, loss_fn, 'coherent_hmc', 200, 
            train_data, train.X, train.Y, valid.X, valid.Y, test.X, test.Y)